In [ ]:
#Step (1) create midline through sample site based on corrdinates provided 
#import arcpy package
import arcpy
#corrdinates selcted for gilgrass sampling site
coordinates = [(-96.0508767, 36.2147766),
               (-96.0518854, 36.2142841)]

# Create a feature class (or layer) with a spatial reference of GCS WGS 1984(common world sphere coord system)
result = arcpy.management.CreateFeatureclass(arcpy.env.scratchGDB, "GilgrassMidLine", "POLYLINE", spatial_reference=4326)
feature_class = result[0]

# Write feature (or Polyline) to new feature class
with arcpy.da.InsertCursor(feature_class, ['SHAPE@']) as cursor:
    cursor.insertRow([coordinates])

In [ ]:
# Set variables for needed features to generate transects using builtin GIS function
in_features = 'GilgrassMidLine'
out_fc_1 = 'Gilgrass_transects'

# Execute GeneratePointsAlongLines by distance
arcpy.GenerateTransectsAlongLines_management(in_features, out_fc_1, '10 Meters',
                                             '100 meters', 'END_POINTS')

In [ ]:
#Now that transects are generated, need to add empty coloumn to my attributes table
#first have ARC recognized that the transects is the object being referenced 
arcpy.env.outputCoordinateSystem = arcpy.Describe("Gilgrass_transects").spatialReference
#set variables 
in_feature = "Gilgrass_transects"
fieldName2 = "Lat_Start"
fieldName3 = "Long_Start"
fieldName4 = "Lat_End"
fieldName5 = "Long_End"
#have the varibles fill table with "double" as text type allowed in element (default for coord.)
arcpy.AddField_management(in_feature, fieldName2, "DOUBLE")
arcpy.AddField_management(in_feature, fieldName3, "DOUBLE")
arcpy.AddField_management(in_feature, fieldName4, "DOUBLE")
arcpy.AddField_management(in_feature, fieldName5, "DOUBLE")
#Arc now pulls the table column names and populates them with x,y coord. at the begining and end of each transect line
arcpy.CalculateGeometryAttributes_management(in_feature, [["Lat_Start", "LINE_START_X"],
                                                           ["Long_Start", "LINE_START_Y"],
                                                           ["Lat_End", "LINE_END_X"],
                                                           ["Long_End", "LINE_END_Y"]], "METERS")

In [ ]:
#Now want to make a stand alone table feature
#Set variables to be converted
inTable = "Gilgrass_transects.dbf"
outLocation = "D:\ARCgis\ArcGISPro\Projects\MyProject\fieldsite_coord"
outTable = "Gilgrass_coord"

# Execute TableToTable
arcpy.TableToTable_conversion(inTable, outLocation, outTable)

In [ ]:
#Final step. Output stand alone table to an excel file
# Set variables
in_table = "Gilgrass_coord"
out_xls = "Gilgrass_coord.xls"

# Execute TableToExcel
arcpy.TableToExcel_conversion(in_table, out_xls)